# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [14]:
from xml.etree import ElementTree as ET

import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [15]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [28]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)
    

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [32]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [18]:
document = ET.parse( './data/mondial_database.xml' )



<Element 'mondial' at 0x7f670801bbd8>

### Question 1

In [77]:
#empty list
cr = []
#loop over countries, leave out missing data
for co in document.iterfind('country'):    
    if co.find('infant_mortality') is not None:
        im = co.find('infant_mortality').text
        co_name = co.find('name').text
        cr.append([co_name, float(im)])
        
#convert to data frame and sort, print top ten
cr_df = pd.DataFrame(cr, columns = ['country', 'infant_mortality_rate'])
                          
cr_df.sort_values('infant_mortality_rate').head(10)


,country,infant_mortality_rate
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


### Question 2

In [85]:
#create empty list 
pop = []
#loop over countries, leave out missing data
#have to do contries first, starting with cities does not work
for co in document.iterfind('country'): 
    co_name = co.find('name').text
    for cy in co.getiterator('city'):
        city = cy.find('name').text
        # exclude those that have no data for city population
        if cy.find('population') is not None:
            pop1 = cy.find('population').text
            pop.append([co_name, city, float(pop1)])
# convert to dataframe
city_df = pd.DataFrame(city_pop, columns=['country', 'city', 'population'])
# sort descending by city population and print
city_df.sort_values('population', ascending=False).head(10)

,country,city,population
176,Germany,Cottbus,99984
519,Russia,Perm,999157
173,Germany,Erlangen,99808
2752,Algeria,Laghouat,99536
2461,Brazil,Alagoinhas,99508
3035,Mozambique,Xai-Xai,99442
2539,Brazil,Camaragibe,99407
2857,Nigeria,Kaduna,993642
56,France,Nancy,99351
486,Russia,Kazan,992675


### Question 3

In [123]:
#empty list
eth = []
#extract data and add to list
for co in document.iterfind('country'):
    co_name = co.find('name').text
    for egroup in co.iterfind('ethnicgroup'):
        co_pop = co.find('population').text
        ethnic_group = egroup.text
        eth_percent = egroup.attrib['percentage']
        eth.append([co_name, ethnic_group, int(co_pop), float(eth_percent)])

#convert list to dataframe
eth_df = pd.DataFrame(eth, columns = ['country', 'eth_group', 'co_pop', 'eth_percent'])

#multiply country population by percentage for each ethnic group
eth_pop = pd.Series(((eth_df.eth_percent/100) * eth_df.co_pop).astype('int'))
eth_df = eth_df.assign(eth_pop = eth_pop.values)

#remove unneeded columns
del eth_df['co_pop']
del eth_df['eth_percent']

#summarize, sort, and print top ten
eth_df.groupby('eth_group').sum().sort_values('eth_pop',ascending = False).head(10)


,eth_pop
eth_group,
Han Chinese,497555113
European,192865792
Indo-Aryan,171645355
Russian,92758431
African,86329356
Japanese,81706273
German,66232183
Dravidian,59599081
English,42314986


### Question 4

In [153]:
#longest river

#empty list to store data
riv_max = []
for ri in document.iterfind('river'):
    if ri.find('length') is not None and ri.find('length').text is not None: #missing values generate error
        river = ri.find('name').text
        length = ri.find('length').text
        co_name = ri.attrib['country']
        riv_max.append([river, co_name, float(length)])

#convert output to datafram
riv_df = pd.DataFrame(riv_max, columns = ['river', 'country', 'length'])

#sort and print max
riv_df.sort_values('length', ascending=False).head(1)




,river,country,length
174,Amazonas,CO BR PE,6448.0


In [154]:
#largest lake

#empty list to store data
lake_max = []
for la in document.iterfind('lake'):
    if la.find('area') is not None and la.find('area').text is not None: #missing values generate error
        lake = la.find('name').text
        area = la.find('area').text
        co_name = la.attrib['country']
        lake_max.append([lake, co_name, float(area)])

#convert output to datafram
lake_df = pd.DataFrame(lake_max, columns = ['lake', 'country', 'area'])

#sort and print max
lake_df.sort_values('area', ascending=False).head(1)




,lake,country,area
54,Caspian Sea,R AZ KAZ IR TM,386400.0


In [155]:
#highest airport

#empty list to store data
air_max = []
for air in document.iterfind('airport'):
    if air.find('elevation') is not None and air.find('elevation').text is not None: #missing values generate error
        airport = air.find('name').text
        elevation = air.find('elevation').text
        co_name = air.attrib['country']
        air_max.append([airport, co_name, float(elevation)])
                        
#convert output to datafram
air_df = pd.DataFrame(air_max, columns = ['airport', 'country', 'elevation'])

#sort and print max
air_df.sort_values('elevation', ascending=False).head(1)



,airport,country,elevation
80,El Alto Intl,BOL,4063.0
